In [1]:
import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import sklearn.datasets as data
from sklearn.datasets import load_iris
import tempfile
%matplotlib inline

In [2]:
data=load_iris()

In [3]:
#X=data.data
#Y=data.target
#data=pd.DataFrame(X,columns=data.feature_names)
#print data.head(10)
X_frame=pd.DataFrame(data.data,columns=data.feature_names)
Y_frame=pd.DataFrame(data.target,columns=['Target'])
data_frame1=pd.concat([X_frame,Y_frame],axis=1)
Y_frame.shape,X_frame.shape
data_frame1['Target'].value_counts()

2    50
1    50
0    50
Name: Target, dtype: int64

In [4]:
Y_frame['Target1']= Y_frame['Target'].apply(lambda x: 1 if x==0 else 0)
data_frame2=pd.concat([X_frame,Y_frame[['Target1']]],axis=1)

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 0 hours 34 mins
H2O cluster version:,3.14.0.1
H2O cluster version age:,"1 year, 4 months and 14 days !!!"
H2O cluster name:,H2O_from_python_kodamsri_w5m92i
H2O cluster total nodes:,1
H2O cluster free memory:,3.271 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [8]:
h2oframe=h2o.H2OFrame(data_frame2,destination_frame='iris')
h2oframe.summary()

Parse progress: |█████████████████████████████████████████████████████████| 100%


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target1
type,real,real,real,real,int
mins,4.3,2.0,1.0,0.1,0.0
mean,5.84333333333,3.054,3.75866666667,1.19866666667,0.333333333333
maxs,7.9,4.4,6.9,2.5,1.0
sigma,0.828066127978,0.433594311362,1.76442041995,0.763160741701,0.47298376984
zeros,0,0,0,0,100
missing,0,0,0,0,0
0,5.1,3.5,1.4,0.2,1.0
1,4.9,3.0,1.4,0.2,1.0
2,4.7,3.2,1.3,0.2,1.0


In [114]:
h2oframe.as_data_frame().describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target1
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,0.333333
std,0.828066,0.433594,1.764420,0.763161,0.472984
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,0.000000
75%,6.400000,3.300000,5.100000,1.800000,1.000000
max,7.900000,4.400000,6.900000,2.500000,1.000000


In [129]:
train,valid,test=h2oframe.split_frame(ratios=[0.8,0.1],destination_frames=['iris_train','iris_valid','iris_test'],seed=200)

In [116]:
print str(train.nrows)+'/'+str(valid.nrows)+'/'+str(test.nrows)

117/22/11


In [119]:
y='Target1'
ignoreFields=[y]
x=[i for i in train.names if i not in ignoreFields]

In [139]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
model=H2OGradientBoostingEstimator(model_id='first',nfolds=9)
model.train(x,y,train,validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [140]:
print model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  first


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 9.35691620778e-06
RMSE: 0.00305890768213
LogLoss: 0.00292219919851
Mean Per-Class Error: 0.0
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.995807058255: 


,0,1,Error,Rate
0,77.0,0.0,0.0,(0.0/77.0)
1,0.0,40.0,0.0,(0.0/40.0)
Total,77.0,40.0,0.0,(0.0/117.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.9958071,1.0,0.0
max f2,0.9958071,1.0,0.0
max f0point5,0.9958071,1.0,0.0
max accuracy,0.9958071,1.0,0.0
max precision,0.9958071,1.0,0.0
max recall,0.9958071,1.0,0.0
max specificity,0.9958071,1.0,0.0
max absolute_mcc,0.9958071,1.0,0.0
max min_per_class_accuracy,0.9958071,1.0,0.0
max mean_per_class_accuracy,0.9958071,1.0,0.0


Gains/Lift Table: Avg response rate: 34.19 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid
accuracy,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
auc,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
err,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
err_count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f0point5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f2,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
lift_top_group,3.846208,1.3900795,2.6,7.5,6.0,2.0,3.3333333,2.5714285,1.7777778,2.8333333,6.0
logloss,0.0029255,0.0001704,0.0029959,0.0026635,0.0026232,0.0032241,0.0028517,0.0029989,0.0033635,0.0029397,0.0026694
max_per_class_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-12-23 23:10:41,0.937 sec,0.0,0.4743397,0.6422748,0.5,1.0,0.6581197,0.4175886,0.5374460,0.5,1.0,0.8181818
,2018-12-23 23:10:41,0.943 sec,1.0,0.4260472,0.5473389,1.0,2.925,0.0,0.3756435,0.4638320,1.0,5.5,0.0
,2018-12-23 23:10:41,0.949 sec,2.0,0.3833846,0.4727753,1.0,2.925,0.0,0.3383838,0.4043099,1.0,5.5,0.0
,2018-12-23 23:10:41,0.954 sec,3.0,0.3453997,0.4120364,1.0,2.925,0.0,0.3050946,0.3548137,1.0,5.5,0.0
,2018-12-23 23:10:41,0.958 sec,4.0,0.3114320,0.3614177,1.0,2.925,0.0,0.2752554,0.3129239,1.0,5.5,0.0
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2018-12-23 23:10:42,1.146 sec,46.0,0.0045638,0.0043632,1.0,2.925,0.0,0.0040467,0.0038981,1.0,5.5,0.0
,2018-12-23 23:10:42,1.152 sec,47.0,0.0041294,0.0039470,1.0,2.925,0.0,0.0036615,0.0035264,1.0,5.5,0.0
,2018-12-23 23:10:42,1.156 sec,48.0,0.0037363,0.0035706,1.0,2.925,0.0,0.0033130,0.0031902,1.0,5.5,0.0
,2018-12-23 23:10:42,1.163 sec,49.0,0.0033807,0.0032301,1.0,2.925,0.0,0.0029976,0.0028861,1.0,5.5,0.0



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
petal length (cm),131.0551147,1.0,0.9340675
petal width (cm),9.2507114,0.0705864,0.0659325
sepal length (cm),0.0,0.0,0.0
sepal width (cm),0.0,0.0,0.0


In [120]:
x

[u'sepal length (cm)',
 u'sepal width (cm)',
 u'petal length (cm)',
 u'petal width (cm)']

In [134]:
model.predict(test),test

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,0.00419294,0.995807
1,0.00419294,0.995807
1,0.00419294,0.995807
1,0.00419294,0.995807
1,0.00419294,0.995807
1,0.00419294,0.995807
0,0.997745,0.00225495
0,0.997745,0.00225495
0,0.997745,0.00225495
0,0.997745,0.00225495


sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target1
5.7,4.4,1.5,0.4,1
5.7,3.8,1.7,0.3,1
5.4,3.4,1.7,0.2,1
4.7,3.2,1.6,0.2,1
4.8,3.1,1.6,0.2,1
5.1,3.8,1.9,0.4,1
5.5,2.3,4,1.3,0
6,2.7,5.1,1.6,0
5.5,2.5,4,1.3,0
5.8,2.6,4,1.2,0


(, )

In [128]:
h2oframe['Target1']=h2oframe['Target1'].asfactor()

In [141]:
model.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 1.19007757182e-05
RMSE: 0.00344975009503
LogLoss: 0.00331800133246
Mean Per-Class Error: 0.0
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.995807058056: 


,0,1,Error,Rate
0,5.0,0.0,0.0,(0.0/5.0)
1,0.0,6.0,0.0,(0.0/6.0)
Total,5.0,6.0,0.0,(0.0/11.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.9958071,1.0,0.0
max f2,0.9958071,1.0,0.0
max f0point5,0.9958071,1.0,0.0
max accuracy,0.9958071,1.0,0.0
max precision,0.9958071,1.0,0.0
max recall,0.9958071,1.0,0.0
max specificity,0.9958071,1.0,0.0
max absolute_mcc,0.9958071,1.0,0.0
max min_per_class_accuracy,0.9958071,1.0,0.0
max mean_per_class_accuracy,0.9958071,1.0,0.0


Gains/Lift Table: Avg response rate: 54.55 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.5454545,0.9958071,1.8333333,1.8333333,1.0,1.0,1.0,1.0,83.3333333,83.3333333
,2,1.0,0.0022549,0.0,1.0,0.0,0.5454545,0.0,1.0,-100.0,0.0


In [10]:
from h2o.estimators import H2OGeneralizedLinearEstimator

In [11]:
prostate = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/smalldata/prostate/prostate.csv")

ValueError: ImportFiles of https://h2o-public-test-data.s3.amazonaws.com/smalldata/prostate/prostate.csv failed on [u'https://h2o-public-test-data.s3.amazonaws.com/smalldata/prostate/prostate.csv']

In [12]:
help(H2OGeneralizedLinearEstimator)

Help on class H2OGeneralizedLinearEstimator in module h2o.estimators.glm:

class H2OGeneralizedLinearEstimator(h2o.estimators.estimator_base.H2OEstimator)
 |  Generalized Linear Modeling
 |  
 |  Fits a generalized linear model, specified by a response variable, a set of predictors, and a
 |  description of the error distribution.
 |  
 |  A subclass of :class:`ModelBase` is returned. The specific subclass depends on the machine learning task
 |  at hand (if it's binomial classification, then an H2OBinomialModel is returned, if it's regression then a
 |  H2ORegressionModel is returned). The default print-out of the models is shown, but further GLM-specific
 |  information can be queried out of the object. Upon completion of the GLM, the resulting object has
 |  coefficients, normalized coefficients, residual/null deviance, aic, and a host of model metrics including
 |  MSE, AUC (for logistic regression), degrees of freedom, and confusion matrices.
 |  
 |  Method resolution order:
 |  